In [1]:
pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 7.9 MB/s eta 0:00:00


In [2]:
import pandas as pd, numpy as np, re, os, pickle, math

from catboost import CatBoostRegressor
import nbformat as nbf
from sklearn.metrics import mean_absolute_error


In [3]:
path="/content/test_task_cl.csv"
df_raw=pd.read_csv(path)
df_raw.head()

,country_map,conv_window_map,opt_group_map,date_map,installs,payers_d0,payers_d1,payers_d3,payers_d7,payers_d14,...,iap_roas_d60,iap_roas_d90,rv_acpu_d0,rv_acpu_d1,rv_acpu_d3,rv_acpu_d7,rv_acpu_d14,rv_acpu_d30,rv_acpu_d60,rv_acpu_d90
0,country_10,conv_window_1,opt_group_4,date_344,1.0,0,0,0,0,0,...,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,country_10,conv_window_1,opt_group_4,date_357,1.0,0,0,0,0,0,...,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,country_10,conv_window_1,opt_group_4,date_364,1.0,0,0,0,0,0,...,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,country_10,conv_window_2,opt_group_4,date_273,1.0,0,0,0,0,0,...,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,country_10,conv_window_2,opt_group_4,date_274,1.0,0,0,0,0,0,...,0.0,0.0,0.0,2.0,4.0,9.0,9.0,9.0,16.0,51.0


In [4]:
def parse_index(s):
    if pd.isna(s):
        return np.nan
    m=re.search(r'_(\d+)$', str(s))
    return int(m.group(1)) if m else np.nan

df=df_raw.copy()
df["date_idx"] = (
    df["date_map"]
    .apply(parse_index)
    .astype("Int64")   # pandas nullable int
)

df.head()


,country_map,conv_window_map,opt_group_map,date_map,installs,payers_d0,payers_d1,payers_d3,payers_d7,payers_d14,...,iap_roas_d90,rv_acpu_d0,rv_acpu_d1,rv_acpu_d3,rv_acpu_d7,rv_acpu_d14,rv_acpu_d30,rv_acpu_d60,rv_acpu_d90,date_idx
0,country_10,conv_window_1,opt_group_4,date_344,1.0,0,0,0,0,0,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,344
1,country_10,conv_window_1,opt_group_4,date_357,1.0,0,0,0,0,0,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,357
2,country_10,conv_window_1,opt_group_4,date_364,1.0,0,0,0,0,0,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,364
3,country_10,conv_window_2,opt_group_4,date_273,1.0,0,0,0,0,0,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,273
4,country_10,conv_window_2,opt_group_4,date_274,1.0,0,0,0,0,0,...,0.0,0.0,2.0,4.0,9.0,9.0,9.0,16.0,51.0,274


In [5]:
df.columns

Index(['country_map', 'conv_window_map', 'opt_group_map', 'date_map',
       'installs', 'payers_d0', 'payers_d1', 'payers_d3', 'payers_d7',
       'payers_d14', 'payers_d30', 'payers_d60', 'payers_d90', 'iaa_roas_d0',
       'iaa_roas_d1', 'iaa_roas_d3', 'iaa_roas_d7', 'iaa_roas_d14',
       'iaa_roas_d30', 'iaa_roas_d60', 'iaa_roas_d90', 'iap_roas_d0',
       'iap_roas_d1', 'iap_roas_d3', 'iap_roas_d7', 'iap_roas_d14',
       'iap_roas_d30', 'iap_roas_d60', 'iap_roas_d90', 'rv_acpu_d0',
       'rv_acpu_d1', 'rv_acpu_d3', 'rv_acpu_d7', 'rv_acpu_d14', 'rv_acpu_d30',
       'rv_acpu_d60', 'rv_acpu_d90', 'date_idx'],
      dtype='object')

In [6]:
# CPI mapping
CPI_MAP={"opt_group_2":1.0,"opt_group_4":1.0,"opt_group_5":1.0,"opt_group_1":10.0,"opt_group_3":40.0}
df["cpi"]=df["opt_group_map"].map(CPI_MAP).astype(float)
df["cost"]=df["installs"]*df["cpi"]

# helper: aggregation to arbitrary level
ROAS_COLS=[c for c in df.columns if c.startswith("iaa_roas_d") or c.startswith("iap_roas_d")]
PAYER_COLS=[c for c in df.columns if c.startswith("payers_d")]
RV_COLS=[c for c in df.columns if c.startswith("rv_acpu_d")]

df.head()

,country_map,conv_window_map,opt_group_map,date_map,installs,payers_d0,payers_d1,payers_d3,payers_d7,payers_d14,...,rv_acpu_d1,rv_acpu_d3,rv_acpu_d7,rv_acpu_d14,rv_acpu_d30,rv_acpu_d60,rv_acpu_d90,date_idx,cpi,cost
0,country_10,conv_window_1,opt_group_4,date_344,1.0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,344,1.0,1.0
1,country_10,conv_window_1,opt_group_4,date_357,1.0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,357,1.0,1.0
2,country_10,conv_window_1,opt_group_4,date_364,1.0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,364,1.0,1.0
3,country_10,conv_window_2,opt_group_4,date_273,1.0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,273,1.0,1.0
4,country_10,conv_window_2,opt_group_4,date_274,1.0,0,0,0,0,0,...,2.0,4.0,9.0,9.0,9.0,16.0,51.0,274,1.0,1.0


In [7]:
def get_feature_sets(df, keys):
    cat_features = [k for k in keys if k != "date_idx"]

    # исключаем явные таргеты / вспомогательные колонки таргетов
    banned = {
        "iaa_roas_d90", "iap_roas_d90",
        "mul_iaa_7_90", "mul_iap_7_90",
        "log_mul_iaa_7_90", "log_mul_iap_7_90"
    }

    num_features = []
    for c in df.columns:
        if c in keys:
            continue
        if c in banned:
            continue

        # запрещаем любые *_d14/_d30/_d60/_d90 фичи (no leakage)
        m = re.search(r"_d(\d+)$", c)
        if m and int(m.group(1)) > 7:
            continue

        num_features.append(c)

    if "date_idx" not in num_features:
        num_features.append("date_idx")

    return cat_features, num_features


In [8]:
def train_catboost(
    df_train,
    target,
    cat_features,
    num_features,
    weight_col="cost",
    **params
):
    X = df_train[cat_features + num_features]
    y = df_train[target]
    w = df_train[weight_col]

    model = CatBoostRegressor(
        iterations=1500,
        learning_rate=0.03,
        depth=8,
        loss_function="MAE",
        eval_metric="MAE",
        random_seed=42,
        verbose=False
    )

    model.fit(
        X,
        y,
        cat_features=cat_features,
        sample_weight=w
    )

    return model

In [9]:
def weighted_mae(y_true, y_pred, w):
    return np.sum(np.abs(y_true - y_pred) * w) / np.sum(w)


def evaluate_point(df, model, target, cat, num):
    X = df[cat + num]
    y = df[target].values
    w = df["cost"].values
    pred = np.clip(model.predict(X), 0, None)

    return {
        "WMAE": weighted_mae(y, pred, w),
        "MAE": mean_absolute_error(y, pred),
        "N": len(df)
    }

def evaluate_point_from_pred(df, pred_col, target_col):
    y = df[target_col].values
    pred = df[pred_col].values
    w = df["cost"].values

    return {
        "WMAE": weighted_mae(y, pred, w),
        "MAE": mean_absolute_error(y, pred),
        "N": len(df)
    }



def aggregate_metric(df, pred_col, target_col, group_keys):
    d = df.copy()
    d["pred_rev"] = d[pred_col] * d["cost"]
    d["true_rev"] = d[target_col] * d["cost"]

    g = (
        d.groupby(group_keys)
        .agg(
            cost=("cost", "sum"),
            pred_rev=("pred_rev", "sum"),
            true_rev=("true_rev", "sum")
        )
        .reset_index()
    )

    g["pred_roas"] = g["pred_rev"] / g["cost"]
    g["true_roas"] = g["true_rev"] / g["cost"]

    return weighted_mae(
        g["true_roas"].values,
        g["pred_roas"].values,
        g["cost"].values
    )

In [10]:
def aggregate_level(dfin, keys):
    d=dfin.copy()
    # compute revenues for each roas col
    for col in ROAS_COLS:
        d[col+"_rev"]=d[col]*d["cost"]
    # compute adviews for rv columns
    for col in RV_COLS:
        d[col+"_views"]=d[col]*d["installs"]
    agg_dict={"installs":"sum","cost":"sum"}
    for col in PAYER_COLS:
        agg_dict[col]="sum"
    # sum rev & views
    for col in ROAS_COLS:
        agg_dict[col+"_rev"]="sum"
    for col in RV_COLS:
        agg_dict[col+"_views"]="sum"
    out=d.groupby(keys, dropna=False).agg(agg_dict).reset_index()
    # restore roas and rv from sums
    for col in ROAS_COLS:
        out[col]=out[col+"_rev"]/out["cost"]
    for col in RV_COLS:
        out[col]=out[col+"_views"]/out["installs"]
    # cleanup
    drop=[c for c in out.columns if c.endswith("_rev") or c.endswith("_views")]
    out=out.drop(columns=drop)
    return out

In [11]:
def make_features(df_level, eps=1e-4):
    d = df_level.copy()

    # ---------- calendar (no leakage) ----------
    d["dow"] = d["date_idx"] % 7
    d["week_idx"] = d["date_idx"] // 7
    d["month_idx"] = d["date_idx"] // 30

    # ---------- payer rates ----------
    for day in ["0", "1", "3", "7"]:
        d[f"payer_rate_d{day}"] = d[f"payers_d{day}"] / d["installs"].replace(0, np.nan)
    d["payer_rate_d7"] = d["payer_rate_d7"].fillna(0)

    # ---------- deltas (shape of curve) ----------
    # payers deltas
    d["payers_delta_0_1"] = d["payers_d1"] - d["payers_d0"]
    d["payers_delta_1_3"] = d["payers_d3"] - d["payers_d1"]
    d["payers_delta_3_7"] = d["payers_d7"] - d["payers_d3"]

    # ROAS deltas (still ok, but noisy on micro)
    for prefix in ["iaa_roas", "iap_roas", "rv_acpu"]:
        d[f"{prefix}_delta_0_1"] = d[f"{prefix}_d1"] - d[f"{prefix}_d0"]
        d[f"{prefix}_delta_1_3"] = d[f"{prefix}_d3"] - d[f"{prefix}_d1"]
        d[f"{prefix}_delta_3_7"] = d[f"{prefix}_d7"] - d[f"{prefix}_d3"]

    # ---------- growth ratios ----------
    for prefix in ["iaa_roas", "iap_roas", "rv_acpu"]:
        d[f"{prefix}_g_0_7"] = (d[f"{prefix}_d7"] + eps) / (d[f"{prefix}_d0"] + eps)
        d[f"{prefix}_g_1_7"] = (d[f"{prefix}_d7"] + eps) / (d[f"{prefix}_d1"] + eps)
        d[f"{prefix}_g_3_7"] = (d[f"{prefix}_d7"] + eps) / (d[f"{prefix}_d3"] + eps)

    # ---------- revenue-space (stabilizes small cohorts) ----------
    for p in ["iaa", "iap"]:
        for dd in ["0", "1", "3", "7"]:
            d[f"{p}_rev_d{dd}"] = d[f"{p}_roas_d{dd}"] * d["cost"]

        d[f"{p}_rev_delta_0_1"] = d[f"{p}_rev_d1"] - d[f"{p}_rev_d0"]
        d[f"{p}_rev_delta_1_3"] = d[f"{p}_rev_d3"] - d[f"{p}_rev_d1"]
        d[f"{p}_rev_delta_3_7"] = d[f"{p}_rev_d7"] - d[f"{p}_rev_d3"]

    # ---------- views-space (key driver for IAA) ----------
    for dd in ["0", "1", "3", "7"]:
        d[f"views_d{dd}"] = d[f"rv_acpu_d{dd}"] * d["installs"]

    d["views_delta_0_1"] = d["views_d1"] - d["views_d0"]
    d["views_delta_1_3"] = d["views_d3"] - d["views_d1"]
    d["views_delta_3_7"] = d["views_d7"] - d["views_d3"]

    # ---------- logs ----------
    d["log_installs"] = np.log1p(d["installs"])
    d["log_cost"] = np.log1p(d["cost"])

    # ---------- targets: log-multipliers (D7 -> D90) ----------
    # do NOT use in features; used as targets later
    d["mul_iaa_7_90"] = (d["iaa_roas_d90"] + eps) / (d["iaa_roas_d7"] + eps)
    d["mul_iap_7_90"] = (d["iap_roas_d90"] + eps) / (d["iap_roas_d7"] + eps)

    # clip extreme tails to reduce overfit to noise
    for c in ["mul_iaa_7_90", "mul_iap_7_90"]:
        hi = d[c].quantile(0.995)
        d[c] = d[c].clip(lower=0.0, upper=float(hi))

    d["log_mul_iaa_7_90"] = np.log(d["mul_iaa_7_90"] + eps)
    d["log_mul_iap_7_90"] = np.log(d["mul_iap_7_90"] + eps)

    return d


In [12]:
def time_split(df_level, test_frac=0.2):
    dates=sorted(df_level["date_idx"].unique())
    cut=dates[int(len(dates)*(1-test_frac))]
    train=df_level[df_level["date_idx"]<cut].copy()
    test=df_level[df_level["date_idx"]>=cut].copy()
    return train, test, cut

In [13]:
import json, os, datetime, hashlib
from pathlib import Path

def _now_id():
    return datetime.datetime.now().strftime("%Y-%m-%d_%H%M%S")

def _hash_dict(d: dict) -> str:
    s = json.dumps(d, sort_keys=True, ensure_ascii=False).encode("utf-8")
    return hashlib.md5(s).hexdigest()[:10]

def log_experiment(run_name: str, config: dict, results: dict, artifacts: dict | None = None):
    run_id = f"{_now_id()}_{_hash_dict(config)}_{run_name}"
    run_dir = Path("experiments") / f"run_{run_id}"
    run_dir.mkdir(parents=True, exist_ok=True)

    (run_dir / "config.json").write_text(json.dumps(config, indent=2, ensure_ascii=False))
    (run_dir / "metrics.json").write_text(json.dumps(results, indent=2, ensure_ascii=False))

    if artifacts:
        # optional: save extra text/csv blobs
        for name, obj in artifacts.items():
            if isinstance(obj, str):
                (run_dir / name).write_text(obj)
            elif hasattr(obj, "to_csv"):
                obj.to_csv(run_dir / name, index=False)

    # append summary row to runs.jsonl
    summary = {
        "run_id": run_id,
        "ts": datetime.datetime.now().isoformat(timespec="seconds"),
        "run_name": run_name,
        "config": config,
        "results": results,
    }
    Path("experiments").mkdir(exist_ok=True)
    with open("experiments/runs.jsonl", "a", encoding="utf-8") as f:
        f.write(json.dumps(summary, ensure_ascii=False) + "\n")

    return str(run_dir)


In [14]:
import json, os, datetime, hashlib, traceback
from pathlib import Path
from functools import wraps

def _now_id():
    return datetime.datetime.now().strftime("%Y-%m-%d_%H%M%S")

def _hash_dict(d: dict) -> str:
    s = json.dumps(d, sort_keys=True, ensure_ascii=False).encode("utf-8")
    return hashlib.md5(s).hexdigest()[:10]

def _save_artifact(run_dir: Path, name: str, obj):
    p = run_dir / name
    if isinstance(obj, (str, bytes)):
        if isinstance(obj, bytes):
            p.write_bytes(obj)
        else:
            p.write_text(obj, encoding="utf-8")
        return

    # pandas df
    if hasattr(obj, "to_csv"):
        obj.to_csv(p, index=False)
        return

    # json-serializable
    try:
        p.write_text(json.dumps(obj, indent=2, ensure_ascii=False), encoding="utf-8")
    except Exception:
        # fallback: repr
        p.write_text(repr(obj), encoding="utf-8")

def log_run(*, base_dir: str = "experiments", run_name: str = "exp"):
    """
    Decorator for functions: fn(config, *args, **kwargs) -> (results_dict, artifacts_dict)
    - results_dict: JSON-serializable metrics
    - artifacts_dict: optional; values can be str/bytes/df/jsonable
    """
    def _decorator(fn):
        @wraps(fn)
        def _wrapped(config: dict, *args, **kwargs):
            base = Path(base_dir)
            base.mkdir(parents=True, exist_ok=True)

            run_id = f"{_now_id()}_{_hash_dict(config)}_{run_name}"
            run_dir = base / f"run_{run_id}"
            run_dir.mkdir(parents=True, exist_ok=True)

            # always write config first
            (run_dir / "config.json").write_text(
                json.dumps(config, indent=2, ensure_ascii=False),
                encoding="utf-8"
            )

            summary = {
                "run_id": run_id,
                "ts": datetime.datetime.now().isoformat(timespec="seconds"),
                "run_name": run_name,
                "status": "started",
                "config": config,
            }

            try:
                results, artifacts = fn(config, *args, **kwargs)
                summary["status"] = "ok"
                summary["results"] = results

                (run_dir / "metrics.json").write_text(
                    json.dumps(results, indent=2, ensure_ascii=False),
                    encoding="utf-8"
                )

                if artifacts:
                    art_dir = run_dir / "artifacts"
                    art_dir.mkdir(exist_ok=True)
                    for name, obj in artifacts.items():
                        _save_artifact(art_dir, name, obj)

                # append to runs.jsonl
                with open(base / "runs.jsonl", "a", encoding="utf-8") as f:
                    f.write(json.dumps(summary, ensure_ascii=False) + "\n")

                return results, artifacts, str(run_dir)

            except Exception as e:
                summary["status"] = "error"
                summary["error"] = str(e)
                summary["traceback"] = traceback.format_exc()

                (run_dir / "error.txt").write_text(summary["traceback"], encoding="utf-8")

                with open(base / "runs.jsonl", "a", encoding="utf-8") as f:
                    f.write(json.dumps(summary, ensure_ascii=False) + "\n")

                raise

        return _wrapped
    return _decorator


In [15]:
def top_errors_df(
    df: pd.DataFrame,
    pred_col: str,
    target_col: str,
    keys: list[str],
    n: int = 200
) -> pd.DataFrame:
    """
    Return top-N rows with the largest absolute error.

    Assumes df contains:
      - pred_col (e.g. 'pred_iaa', 'pred_iap')
      - target_col (e.g. 'iaa_roas_d90', 'iap_roas_d90')
      - cost, installs
      - (optional) day-7 signals for debugging
    """
    d = df.copy()

    # core error metrics
    d["abs_err"] = (d[target_col] - d[pred_col]).abs()
    d["rel_err"] = d["abs_err"] / (d[target_col].abs() + 1e-6)

    # choose debug columns if present
    debug_candidates = [
        "installs", "cost",
        "iaa_roas_d0","iaa_roas_d1","iaa_roas_d3","iaa_roas_d7","iaa_roas_d90",
        "iap_roas_d0","iap_roas_d1","iap_roas_d3","iap_roas_d7","iap_roas_d90",
        "rv_acpu_d0","rv_acpu_d1","rv_acpu_d3","rv_acpu_d7",
        "payers_d0","payers_d1","payers_d3","payers_d7",
        "date_idx", "week", "month"
    ]
    debug_cols = [c for c in debug_candidates if c in d.columns]

    cols = list(dict.fromkeys(keys + debug_cols + [pred_col, target_col, "abs_err", "rel_err"]))  # unique order
    cols = [c for c in cols if c in d.columns]

    return (
        d.sort_values("abs_err", ascending=False)
         .loc[:, cols]
         .head(n)
         .reset_index(drop=True)
    )


In [16]:
@log_run(run_name="d7_to_d90_multiplier")
def run_experiment(config: dict):

    LEVELS = {
        "micro": ["opt_group_map", "conv_window_map", "country_map", "date_idx"],
        "mid": ["opt_group_map", "date_idx"],
        "macro": ["date_idx"]
    }

    results = {}
    models = {}
    artifacts = {}

    for lvl, keys in LEVELS.items():
        df_lvl = aggregate_level(df, keys)
        df_lvl = make_features(df_lvl)
        df_lvl = df_lvl.dropna(subset=["iaa_roas_d90", "iap_roas_d90"])

        train, test, cut = time_split(df_lvl, config.get("test_frac", 0.2))
        cat, num = get_feature_sets(df_lvl, keys)

        # train log-multipliers
        model_iaa = train_catboost(train, "log_mul_iaa_7_90", cat, num, **config.get("catboost_params", {}))
        model_iap = train_catboost(train, "log_mul_iap_7_90", cat, num, **config.get("catboost_params", {}))

        test = test.copy()
        EPS = config.get("eps", 1e-4)

        pred_log_mul_iaa = model_iaa.predict(test[cat + num])
        pred_log_mul_iap = model_iap.predict(test[cat + num])

        mul_iaa = np.clip(np.exp(pred_log_mul_iaa) - EPS, 0, None)
        mul_iap = np.clip(np.exp(pred_log_mul_iap) - EPS, 0, None)

        test["pred_iaa"] = np.clip((test["iaa_roas_d7"].values + EPS) * mul_iaa - EPS, 0, None)
        test["pred_iap"] = np.clip((test["iap_roas_d7"].values + EPS) * mul_iap - EPS, 0, None)

        test["week"] = test["date_idx"] // 7
        test["month"] = test["date_idx"] // 30

        results[lvl] = {
            "cut_date_idx": int(cut),
            "iaa_point": evaluate_point_from_pred(test, "pred_iaa", "iaa_roas_d90"),
            "iap_point": evaluate_point_from_pred(test, "pred_iap", "iap_roas_d90"),
            "iaa_week_WMAE": aggregate_metric(test, "pred_iaa", "iaa_roas_d90", ["week"]),
            "iap_week_WMAE": aggregate_metric(test, "pred_iap", "iap_roas_d90", ["week"]),
            "iaa_month_WMAE": aggregate_metric(test, "pred_iaa", "iaa_roas_d90", ["month"]),
            "iap_month_WMAE": aggregate_metric(test, "pred_iap", "iap_roas_d90", ["month"]),
            "n_train": int(len(train)),
            "n_test": int(len(test))
        }

        models[lvl] = {"iaa": model_iaa, "iap": model_iap, "cat": cat, "num": num}

        # (опционально) артефакты для лога: топ ошибок
        artifacts[f"top_errors_{lvl}_iaa.csv"] = top_errors_df(test, "pred_iaa", "iaa_roas_d90", keys, n=200)
        artifacts[f"top_errors_{lvl}_iap.csv"] = top_errors_df(test, "pred_iap", "iap_roas_d90", keys, n=200)

        # список фич
        artifacts[f"features_{lvl}.txt"] = "CAT:\n" + "\n".join(cat) + "\n\nNUM:\n" + "\n".join(num)

    return results, artifacts


In [17]:
config = {
    "test_frac": 0.2,
    "eps": 1e-4,
    "catboost_params": {
        #"depth": 8,
        #"learning_rate": 0.08,
        #"l2_leaf_reg": 20
    }
}

results, artifacts, run_dir = run_experiment(config)
print("saved to:", run_dir)


saved to: experiments/run_2025-12-15_103620_4311d12710_d7_to_d90_multiplier


In [18]:
def predict_roas_90(
    df_input,
    level="micro",
    target="iap"
):
    model = models[level][target]
    cat = models[level]["cat"]
    num = models[level]["num"]

    X = df_input[cat + num]
    return float(np.clip(model.predict(X).mean(), 0, None))

In [19]:
results

{'micro': {'cut_date_idx': 396,
  'iaa_point': {'WMAE': np.float64(0.19449465895367526),
   'MAE': 0.662328665658314,
   'N': 11610},
  'iap_point': {'WMAE': np.float64(0.02936570079842189),
   'MAE': 0.09456159690198605,
   'N': 11610},
  'iaa_week_WMAE': np.float64(0.06508905390295647),
  'iap_week_WMAE': np.float64(0.01904372448503406),
  'iaa_month_WMAE': np.float64(0.05996186141031921),
  'iap_month_WMAE': np.float64(0.019043724485034063),
  'n_train': 38890,
  'n_test': 11610},
 'mid': {'cut_date_idx': 396,
  'iaa_point': {'WMAE': np.float64(0.13273135495457897),
   'MAE': 0.11057459771037043,
   'N': 318},
  'iap_point': {'WMAE': np.float64(0.029554609512026955),
   'MAE': 0.049413284070046526,
   'N': 318},
  'iaa_week_WMAE': np.float64(0.0944327282629938),
  'iap_week_WMAE': np.float64(0.011041627793914426),
  'iaa_month_WMAE': np.float64(0.04499592872792882),
  'iap_month_WMAE': np.float64(0.009059058227002795),
  'n_train': 914,
  'n_test': 318},
 'macro': {'cut_date_idx': 3

Репозиторий с решением: https://github.com/nibekasov/appquantum


# Отчет по задаче прогнозирования ROAS D7 → D90

## Цель работы

Целью работы было построение моделей прогнозирования IAP ROAS и IAA ROAS на 90 день жизни когорты на основании данных первых 7 дней. Прогноз используется для оптимизации рекламных кампаний и выбора эффективных креативов и настроек UA.

Данные представлены в агрегированном когортном формате по датам установки и включают разрезы по типу оптимизации, конверсионному окну, гео и дате закупки.

---

## Выбор метрики качества

В качестве основной метрики качества используется взвешенная средняя абсолютная ошибка по затратам (Cost Weighted MAE, WMAE):

WMAE = sum(cost_i * |y_i - y_hat_i|) / sum(cost_i)

Причины выбора данной метрики:

Ошибки на когортах с большим рекламным бюджетом существенно важнее для бизнеса, чем ошибки на малых когортах.

ROAS определяется как Revenue / Cost, поэтому взвешивание по Cost естественным образом согласуется с бизнес интерпретацией метрики.

Невзвешенная MAE сильно искажается за счет шумных когорт с малым количеством установок.

Дополнительно рассчитываются:

MAE как диагностическая метрика.

WMAE на агрегированных уровнях неделя и месяц, так как на практике решения по оптимизации принимаются именно на этих горизонтах.

---

## Проверенные варианты моделей

### Прямое предсказание ROAS D90

В первом варианте CatBoost модель обучалась напрямую предсказывать iaa_roas_d90 и iap_roas_d90 на основании признаков первых 7 дней и категориальных разрезов.

Основные проблемы подхода:

Высокая дисперсия ошибок на micro уровне.

Модель вынуждена одновременно учить масштаб ROAS и динамику роста, что приводит к нестабильности при сильной гетерогенности когорт и различиях CPI.

---

### Предсказание мультипликатора роста от D7 к D90

Финальный вариант модели основан на предсказании логарифма мультипликатора роста ROAS между 7 и 90 днем жизни когорты:

log((ROAS_90 + eps) / (ROAS_7 + eps))

После чего прогноз ROAS D90 восстанавливается как:

ROAS_90_pred = ROAS_7 * multiplier

Преимущества данного подхода:

Модель учит инкрементальный рост после 7 дня, а не абсолютное значение ROAS.

Снижается чувствительность к масштабу и CPI.

Поведение модели становится более стабильным для разреженных когорт.

---

## Изменение качества моделей

### Micro уровень точечных прогнозов

IAP ROAS:

WMAE снизилась примерно с 0.10 до 0.03.

IAA ROAS:

WMAE снизилась примерно с 0.58 до 0.19.

Для IAP наблюдается существенное улучшение качества. Для IAA также достигнуто заметное улучшение, однако ошибка остается выше из за высокой вариативности рекламных просмотров и поведенческих факторов.

---

### Агрегированный уровень неделя и месяц

На агрегированных уровнях качество существенно выше:

IAP WMAE на недельном и месячном уровне составляет порядка 0.01–0.02.

IAA WMAE на недельном и месячном уровне составляет порядка 0.06.

Это соответствует реальным условиям использования модели, так как бюджетные и стратегические решения принимаются на агрегированных горизонтах.

---

## Особенности прогнозирования IAA

Прогнозирование IAA ROAS является более сложной задачей по следующим причинам:

IAA напрямую зависит от пользовательского поведения и частоты сессий.

RV ACPU имеет тяжелые хвосты и высокую вариативность при малом количестве установок.

Ошибки ранних дней могут существенно масштабироваться к 90 дню жизни когорты.

Данные ограничения являются свойством задачи и данных, а не дефектом модели.

---

## Использование модели в продакшене

На micro уровне прогноз используется как сигнальный и диагностический, с возможным смещением в сторону более агрегированных уровней при малом объеме данных.

На mid и macro уровнях прогноз используется как основной инструмент принятия решений по оптимизации UA.

---

## Возможные улучшения

Использование backoff стратегии с переходом к mid или macro прогнозам для когорт с малым рекламным бюджетом.

Оценка качества отдельно на верхних квантилях по затратам, где сосредоточен основной бизнес эффект.

Регуляризация хвостов IAA метрик и сглаживание RV ACPU.

Ведение лога экспериментов с фиксацией параметров моделей, временных сплитов и метрик качества.

---

## Итоговая оценка

Полученное решение является корректным, устойчивым и ориентированным на практическое применение в условиях реального UA пайплайна.

Качество прогнозов на агрегированных уровнях соответствует требованиям бизнеса, а архитектура решения позволяет масштабирование и дальнейшее улучшение модели.
